# Canada (NHL) Hockey Players

## Libraries

In [1]:
import numpy as np
import pandas as pd
import polars as pl

In [2]:
# import seaborn as sns
# import plotly.express as px 
import hvplot.polars

## Data

In [4]:
data_url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2024/2024-01-09/nhl_rosters.csv"
df_raw = pl.read_csv(
    data_url,
    infer_schema_length=10000,
    ignore_errors = True,
    null_values = list(['NA', '']))
df_raw.head(4)

team_code,season,position_type,player_id,headshot,first_name,last_name,sweater_number,position_code,shoots_catches,height_in_inches,weight_in_pounds,height_in_centimeters,weight_in_kilograms,birth_date,birth_city,birth_country,birth_state_province
str,i64,str,i64,str,str,str,i64,str,str,i64,i64,i64,i64,str,str,str,str
"""ATL""",19992000,"""forwards""",8467867,"""https://assets…","""Bryan""","""Adams""",26,"""L""","""L""",72,185,183,84,"""1977-03-20""","""Fort St. James…","""CAN""","""British Columb…"
"""ATL""",19992000,"""forwards""",8445176,"""https://assets…","""Donald""","""Audette""",28,"""R""","""R""",68,191,173,87,"""1969-09-23""","""Laval""","""CAN""","""Quebec"""
"""ATL""",19992000,"""forwards""",8460014,"""https://assets…","""Eric""","""Bertrand""",83,"""L""","""L""",73,205,185,93,"""1975-04-16""","""St-Ephrem""","""CAN""","""Quebec"""
"""ATL""",19992000,"""forwards""",8460510,"""https://assets…","""Jason""","""Botterill""",28,"""L""","""L""",76,217,193,98,"""1976-05-19""","""Edmonton""","""CAN""","""Alberta"""


In [5]:
df_raw.columns

['team_code',
 'season',
 'position_type',
 'player_id',
 'headshot',
 'first_name',
 'last_name',
 'sweater_number',
 'position_code',
 'shoots_catches',
 'height_in_inches',
 'weight_in_pounds',
 'height_in_centimeters',
 'weight_in_kilograms',
 'birth_date',
 'birth_city',
 'birth_country',
 'birth_state_province']

In [88]:
dfc1 = (df_raw
    .select(
        'team_code', 'season', 'position_type', 'player_id', 'shoots_catches', 
        'height_in_centimeters', 'weight_in_kilograms', 'birth_date', 'birth_city', 'birth_country')
    .rename({
        'team_code' : 'code', 'position_type':'position',  'shoots_catches':'catches', 
        'height_in_centimeters':'height', 'weight_in_kilograms':'weight', 'birth_date':'dob', 
        'birth_city':'city', 'birth_country':'country'})
    .with_columns(
        pl.col('dob').cast(pl.Date()),
        pl.col('season').cast(pl.String()),
        pl.col('player_id').cast(pl.String()))
    .with_columns(
        pl.col('season').str.slice(-8, length =  4).cast(pl.Int64()).alias('startyear'),
        pl.col('season').str.slice(4).cast(pl.Int64()).alias('endyear'),
        pl.col('dob').dt.year().alias('year'), 
        pl.col('dob').dt.strftime('%B').alias('monthname'),
        pl.col('dob').dt.strftime('%A').alias('dayname'),
        pl.col('catches').cast(pl.Categorical))
    .with_columns(
        (pl.col('startyear') - pl.col('year')).alias('startage'))
)


In [89]:
dfc1.shape

(54883, 16)

In [90]:
cols_order = [
     'player_id', 'code', 'season', 'position', 'catches', 'height', 'weight', 'dob', 'city',
    'country', 'startyear', 'endyear', 'year', 'monthname', 'dayname', 'startage']
dfc = dfc1.select(cols_order).unique(subset=["player_id"], keep="first", maintain_order=True)

In [91]:
dfc.shape

(8472, 16)

In [92]:
dfc.head()

player_id,code,season,position,catches,height,weight,dob,city,country,startyear,endyear,year,monthname,dayname,startage
str,str,str,str,cat,i64,i64,date,str,str,i64,i64,i32,str,str,i64
"""8467867""","""ATL""","""19992000""","""forwards""","""L""",183,84,1977-03-20,"""Fort St. James…","""CAN""",1999,2000,1977,"""March""","""Sunday""",22
"""8445176""","""ATL""","""19992000""","""forwards""","""R""",173,87,1969-09-23,"""Laval""","""CAN""",1999,2000,1969,"""September""","""Tuesday""",30
"""8460014""","""ATL""","""19992000""","""forwards""","""L""",185,93,1975-04-16,"""St-Ephrem""","""CAN""",1999,2000,1975,"""April""","""Wednesday""",24
"""8460510""","""ATL""","""19992000""","""forwards""","""L""",193,98,1976-05-19,"""Edmonton""","""CAN""",1999,2000,1976,"""May""","""Wednesday""",23
"""8459596""","""ATL""","""19992000""","""forwards""","""L""",185,98,1973-08-24,"""Sudbury""","""CAN""",1999,2000,1973,"""August""","""Friday""",26


## EDA

### Country

In [96]:
dfc.group_by('country').agg(pl.count()).sort('count', descending=True).head(9).select('country').get_columns()

[shape: (9,)
 Series: 'country' [str]
 [
 	"CAN"
 	"USA"
 	"SWE"
 	"RUS"
 	"CZE"
 	"FIN"
 	"SVK"
 	"GBR"
 	"CHE"
 ]]

In [97]:
top9 = ["CAN", "USA", "SWE", "CZE", "RUS", "FIN", "SVK", "CHE", "GBR"]

In [98]:
country_df = (dfc
            .select('country')
            .with_columns(
                  pl.when(pl.col('country').is_in(top9)).then(pl.col('country')).otherwise(pl.lit('Other')).alias("topcountry"))
            .with_columns(pl.col('topcountry').cast(pl.Categorical()))
              
)
ctop = country_df.group_by('topcountry').agg(pl.count()).sort('count', descending=True)
ctop

topcountry,count
cat,u32
"""CAN""",5468
"""USA""",1403
"""SWE""",403
"""RUS""",287
"""CZE""",258
"""FIN""",256
"""Other""",210
"""SVK""",93
"""GBR""",52


In [99]:
ctop.plot.bar(
    x="topcountry", y="count",
    color="gold", title="Players Origin Country: Top 9", ylabel="Total", xlabel = "")

:Bars   [topcountry]   (count)

In [100]:
ctop2 = (country_df
         .group_by('topcountry').agg(pl.count()).sort('count', descending=False)
         .with_columns(
                  pl.when(pl.col('topcountry') == 'Other').then(pl.lit('yes')).otherwise(pl.lit('no')).alias("ccolor"))
         )
ctop2

topcountry,count,ccolor
cat,u32,str
"""CHE""",42,"""no"""
"""GBR""",52,"""no"""
"""SVK""",93,"""no"""
"""Other""",210,"""yes"""
"""FIN""",256,"""no"""
"""CZE""",258,"""no"""
"""RUS""",287,"""no"""
"""SWE""",403,"""no"""
"""USA""",1403,"""no"""


In [101]:
ctop2.plot.barh(
    x="topcountry", y="count",
    color="teal", title="Players Origin Country: Top 9", ylabel="Total", xlabel = "")

:Bars   [topcountry]   (count)

In [102]:
ctop2.plot.barh(
    x="topcountry", y="count",
    color='topcountry',
    cmap=['orange', 'orange', 'orange',  'blue', 'orange', 'orange', 'orange', 'orange', 'orange', 'orange'],
    title="Players Origin Country: Top 9", ylabel="Total", xlabel = "",
    legend=False)
#.opts(legend_position='top')

:Bars   [topcountry]   (count)

### TEAMs

In [142]:
teamtop = dfc.group_by('code').agg(pl.count()).sort('count', descending=False).head(20)
teamtop.plot.barh(
    x="code", y="count", color = 'forestgreen',
    title="Top 20 Teams", ylabel="Total Players", xlabel = "Teams")

:Bars   [code]   (count)

 ### By Group: CAN - USA -  WORLD

In [105]:
dfc.filter(dfc.select(["player_id"]).is_duplicated())

player_id,code,season,position,catches,height,weight,dob,city,country,startyear,endyear,year,monthname,dayname,startage
str,str,str,str,cat,i64,i64,date,str,str,i64,i64,i32,str,str,i64


In [184]:
gdf = (dfc
    .with_columns(
                  pl.when(pl.col('country').is_in(['CAN', 'USA'])).then(pl.col('country')).otherwise(pl.lit('WORLD')).alias("grpcountry"))
 );

gdf3 = (dfc
    .with_columns(
                  pl.when(pl.col('country').is_in(['CAN', 'USA'])).then(pl.col('country')).otherwise(pl.lit('WORLD')).alias("grpcountry"))
 );

In [128]:
(gdf
    .group_by('grpcountry')
    .agg(pl.col(['height', 'startage']).mean())
    )

grpcountry,height,startage
str,f64,f64
"""WORLD""",185.444722,24.204247
"""USA""",184.363312,24.654312
"""CAN""",182.540842,24.261156


In [132]:
(gdf
    .group_by('grpcountry')
    .agg(pl.col('height', 'weight', 'startage').mean().round(2))
    )

grpcountry,height,weight,startage
str,f64,f64,f64
"""CAN""",182.54,85.99,24.26
"""USA""",184.36,88.95,24.65
"""WORLD""",185.44,89.69,24.2


In [150]:
gdf.plot.violin(
    y='startage', by='grpcountry', c='grpcountry', 
    title = 'Players Age  Distribution At First Season', ylabel='AGE', xlabel = 'GROUPS',
    cmap=['gold', 'silver', 'brown'], legend=False, 
    width=600, height=400, padding=0.2
    )

:Violin   [grpcountry]   (startage)

In [162]:
gdf.plot.violin(
    y='weight', by='grpcountry', c='grpcountry', 
    title = 'Players Weight  Distribution', ylabel='WEIGHT', xlabel = 'GROUPS',
    cmap=['crimson', 'grey', 'navy'], legend=False, 
    width=600, height=400, padding=0.2
    )

:Violin   [grpcountry]   (weight)

In [154]:
gdf.hvplot.box(
    y='startage', by='grpcountry', c='grpcountry', 
    title = 'Players Age  Distribution At First Season', ylabel='AGE', xlabel = 'GROUPS',
    cmap=['gold', 'silver', 'brown'], legend=False, 
    width=600, height=400, padding=0.1
    )

:BoxWhisker   [grpcountry]   (startage)

In [185]:
gdf.plot.box(
    y='height', by='grpcountry', c='grpcountry', 
    title = 'Players Height  Distribution ', ylabel='HEIGHT', xlabel = 'GROUPS',
    cmap=['crimson', 'grey', 'navy'], legend=False, 
    width=600, height=400, padding=0.2
    )

:BoxWhisker   [grpcountry]   (height)

In [211]:
custom_order = ["WORLD",  "USA", "CAN"]
with pl.StringCache():
    pl.Series(custom_order).cast(pl.Categorical)
    gdf2 = gdf.with_columns(pl.col('grpcountry').cast(pl.Categorical('physical')))

In [212]:
gcount1 = gdf.group_by('grpcountry').agg(pl.count()).sort('count', descending=True)
gcount2 = gdf2.group_by('grpcountry').agg(pl.count())
gcount3 = gdf3.group_by('grpcountry').agg(pl.count())

In [182]:
gcount1.plot.bar(
    x = 'grpcountry', y = 'count', color= 'grpcountry', title='Bar Ordered With  Sort',
    cmap= ["#ba2649", "#ffa7ca", "#1a6b54"])

:Bars   [grpcountry]   (count)

In [208]:
gcount2.plot.bar(
    x = 'grpcountry', y = 'count', color= 'grpcountry', title='Bar Ordered With  Cat Column',
    cmap= ["#ba2649", "#ffa7ca", "#1a6b54"])

:Bars   [grpcountry]   (count)

In [209]:
gcount3.plot.bar(
    x = 'grpcountry', y = 'count', color= 'grpcountry', title='Bar Ordered With  Alphabet',
    cmap= ["#ba2649", "#ffa7ca", "#1a6b54"])

:Bars   [grpcountry]   (count)

In [ ]:
#custom_order = ["USA", "WORLD", "CAN"]
#with pl.StringCache():
 #   pl.Series(custom_order).cast(pl.Categorical)
 #   gdf2 = gdf.with_columns(pl.col('grpcountry').cast(pl.Categorical))

In [227]:
poscount = gdf.group_by('position').agg(pl.count())
poscount.plot.bar(
    x = 'position', y = 'count', color= 'position', title='Sorted  By Default (Descending) ',
    cmap= ["#ba2649", "#ffa7ca", "#1a6b54"])

:Bars   [position]   (count)

In [228]:
poscount2 = gdf.group_by('position').agg(pl.count()).sort('count', descending=False)
poscount2.plot.bar(
    x = 'position', y = 'count', color= 'position', title='Sorted Ascending Order',
    cmap= ["#ba2649", "#ffa7ca", "#1a6b54"])

:Bars   [position]   (count)

In [229]:
gdf.shape

(8472, 17)

### Relationship

In [195]:
dfc.plot.scatter(x = 'startage', y = 'height', color= 'position')

:Scatter   [startage]   (height,position)

### Pivot - Melt

In [234]:
long_df = pl.DataFrame(
    {
        "foo": ["one", "one", "two", "two", "one", "two"],
        "bar": ["y", "y", "y", "x", "x", "x"],
        "baz": [1, 2, 3, 4, 5, 6],
    }
)
long_df.pivot(values="baz", index="foo", columns="bar", aggregate_function="sum")

foo,y,x
str,i64,i64
"""one""",3,5
"""two""",3,10


In [231]:
wide_df = pl.DataFrame(
    {
        "a": ["x", "y", "z"],
        "b": [1, 3, 5],
        "c": [2, 4, 6],
    }
)
#import polars.selectors as cs
wide_df.melt(id_vars="a", value_vars=pl.selectors.numeric())

a,variable,value
str,str,i64
"""x""","""b""",1
"""y""","""b""",3
"""z""","""b""",5
"""x""","""c""",2
"""y""","""c""",4
"""z""","""c""",6


In [240]:
num_df = (dfc
          .select('player_id', 'height',	'weight', 'startage' )
          .melt(id_vars="player_id", value_vars=pl.selectors.numeric())
          .rename({"variable": "measure", "value": "vals"})
          )
num_df.head(3)

player_id,measure,vals
str,str,i64
"""8467867""","""height""",183
"""8445176""","""height""",173
"""8460014""","""height""",185


In [241]:
num_df.hvplot.hist("vals", by="measure", subplots=True, width=250)

:NdLayout   [measure]
   :Histogram   [vals]   (vals_count)

In [243]:
dfc.hvplot.hist("weight", title='Players Weight Distribution')

:Histogram   [weight]   (weight_count)